In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="H0Y0sh6WQgJi9NbXK59v")
project = rf.workspace("roboflow-100").project("animals-ij5d2")
version = project.version(2)
dataset = version.download("coco")
                

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.14.0 which is incompatible.


   ---------------------------------------- 0.0/38.8 MB ? eta -:--:--
   -- ------------------------------------- 2.1/38.8 MB 10.7 MB/s eta 0:00:04
   ---- ----------------------------------- 4.5/38.8 MB 11.2 MB/s eta 0:00:04
   ------- -------------------------------- 6.8/38.8 MB 11.7 MB/s eta 0:00:03
   ---------- ----------------------------- 10.0/38.8 MB 12.2 MB/s eta 0:00:03
   -------------- ------------------------- 13.6/38.8 MB 13.2 MB/s eta 0:00:02
   ------------------ --------------------- 17.8/38.8 MB 14.2 MB/s eta 0:00:02
   ---------------------- ----------------- 22.0/38.8 MB 15.1 MB/s eta 0:00:02
   --------------------------- ------------ 26.5/38.8 MB 16.0 MB/s eta 0:00:01
   -------------------------------- ------- 31.2/38.8 MB 16.6 MB/s eta 0:00:01
   ------------------------------------- -- 36.7/38.8 MB 17.5 MB/s eta 0:00:01
   ---------------------------------------- 38.8/38.8 MB 17.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.6 MB ? eta -:-


Extracting Dataset Version Zip to animals-2 in coco:: 100%|██████████| 1008/1008 [00:06<00:00, 167.15it/s]


In [2]:
!pip install transformers datasets torchvision wandb --quiet


In [3]:
import os
import torch
import wandb
import torchvision
import numpy as np
from PIL import Image
from datasets import Dataset, DatasetDict
from transformers import ViTForImageClassification, ViTFeatureExtractor, Trainer, TrainingArguments
import torchvision.transforms as T
from torchvision.datasets import ImageFolder
from sklearn.metrics import accuracy_score


In [4]:
wandb.login()
wandb.init(project="animal-vit", name="vit-animal-classifier")


wandb: Currently logged in as: dhabalia-taksh (dhabalia-taksh-mit-world-peace-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [5]:
feature_extractor = ViTFeatureExtractor.from_pretrained("google/vit-base-patch16-224-in21k")


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

`huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\dhaba\.cache\huggingface\hub\models--google--vit-base-patch16-224-in21k. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.


In [6]:
def transform(example):
    image = Image.open(example['image']).convert("RGB")
    inputs = feature_extractor(images=image, return_tensors="pt")
    inputs['label'] = example['label']
    return {
        'pixel_values': inputs['pixel_values'].squeeze(),
        'label': inputs['label']
    }


In [12]:
import json
from datasets import Dataset
from PIL import Image
from pathlib import Path

def load_coco_classification_dataset(img_dir, coco_json_path):
    # Load COCO annotation file
    with open(coco_json_path, 'r') as f:
        coco = json.load(f)

    # Map image_id to file name
    img_id_to_file = {img['id']: img['file_name'] for img in coco['images']}

    # Map image_id to class_id (first object only, for now)
    img_id_to_class = {}
    for ann in coco['annotations']:
        if ann['image_id'] not in img_id_to_class:
            img_id_to_class[ann['image_id']] = ann['category_id']

    # Map class_id to label name
    class_id_to_name = {cat['id']: cat['name'] for cat in coco['categories']}

    # Create dataset
    data = []
    for img_id, fname in img_id_to_file.items():
        if img_id in img_id_to_class:
            label_id = img_id_to_class[img_id]
            label = class_id_to_name[label_id]
            data.append({
                "image_path": str(Path(img_dir) / fname),
                "label": label
            })

    print(f"Found {len(data)} labeled samples.")
    return Dataset.from_list(data), list(sorted(set(d['label'] for d in data)))

# Example usage for test set:
test_ds, label_names = load_coco_classification_dataset(
    img_dir="animals-2/test",
    coco_json_path="animals-2/test/_annotations.coco.json"
)


Found 100 labeled samples.


In [14]:
def prepare_dataset(path):
    folder = ImageFolder(path)
    image_paths, labels = zip(*folder.samples)
    dataset = Dataset.from_dict({'image': list(image_paths), 'label': list(labels)})
    return dataset.map(transform)

train_ds = prepare_dataset("animals-2/train")
val_ds = prepare_dataset("animals-2/valid")
test_ds = prepare_dataset("animals-2/test")

label_names = ImageFolder("dst/animals-2/train").classes
print("Classes:", label_names)

dataset = DatasetDict({
    'train': train_ds,
    'validation': val_ds,
    'test': test_ds
})


FileNotFoundError: Couldn't find any class folder in animals-2/train.

In [ ]:
model = ViTForImageClassification.from_pretrained(
    "google/vit-base-patch16-224-in21k",
    num_labels=len(label_names),
    id2label={i: c for i, c in enumerate(label_names)},
    label2id={c: i for i, c in enumerate(label_names)}
)


In [ ]:
training_args = TrainingArguments(
    output_dir="./vit_output",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="wandb"  # Enable wandb logging
)


In [ ]:
def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return {"accuracy": accuracy_score(eval_pred.label_ids, predictions)}


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['validation'],
    compute_metrics=compute_metrics,
)
trainer.train()


In [ ]:
metrics = trainer.evaluate(dataset['test'])
print(metrics)


In [ ]:
model.save_pretrained("vit-final-animal")
feature_extractor.save_pretrained("vit-final-animal")
